# [Chapter 5. Files and I/O](http://chimera.labs.oreilly.com/books/1230000000393/ch05.html)

All programs need to perform input and output.  
This chapter covers common idioms for working with different kinds of files, including text and binary files, file encodings, and other related matters.  
Techniques for manipulating filenames and directories are also covered.

## [Reading and Writing Text Data](http://chimera.labs.oreilly.com/books/1230000000393/ch05.html#_reading_and_writing_text_data)

### Problem 

You need to read or write text data, possibly in different text encodings such as ASCII, UTF-8, or UTF-16.

### Solution

Use the `open()` function with mode `rt` to read a text file.

In [14]:
# Read the entire file as a single string:
with open('python_ipsum.txt', 'rt') as f:
    data = f.read()
f.close()
print(data)

Python Ipsum: Your source for Python-flavored placeholder text.
http://pythonipsum.com/

Lambda raspberrypi beautiful test script. Kwargs integration itertools dict reduce egg import cython.

Django integration functools unit object kwargs functools dictionary cython. Cython integration exception. Lambda integration diversity bdfl. Return integration exception self dunder. Python integration mercurial bdfl python lambda generator. Kwargs raspberrypi decorator unit cython import. Cython raspberrypi exception unit future klass exception. Python integration community. Object raspberrypi community bdfl cython import method.

Method raspberrypi diversity 2to3 return yield unit yield guido. Method integration mercurial unit import python exception dictionary. Django raspberrypi functools self import. Python integration mercurial dict return klass. Lambda integration mercurial 2to3 cython zen.

Import raspberrypi community pypi reduce dunder pyladies functools. Lambda raspberrypi decorator bd

In [15]:
# Iterate over the lines of the file:
with open('python_ipsum.txt', 'rt') as f:
    for line in f:
        print(line)
f.close()

Python Ipsum: Your source for Python-flavored placeholder text.

http://pythonipsum.com/



Lambda raspberrypi beautiful test script. Kwargs integration itertools dict reduce egg import cython.



Django integration functools unit object kwargs functools dictionary cython. Cython integration exception. Lambda integration diversity bdfl. Return integration exception self dunder. Python integration mercurial bdfl python lambda generator. Kwargs raspberrypi decorator unit cython import. Cython raspberrypi exception unit future klass exception. Python integration community. Object raspberrypi community bdfl cython import method.



Method raspberrypi diversity 2to3 return yield unit yield guido. Method integration mercurial unit import python exception dictionary. Django raspberrypi functools self import. Python integration mercurial dict return klass. Lambda integration mercurial 2to3 cython zen.



Import raspberrypi community pypi reduce dunder pyladies functools. Lambda raspberrypi dec

Similarly, to write a text file, use `open()` with mode `wt` to write a file, clearing and overwriting the previous contents (if there were any, at least).

In [16]:
# Write chunks of text data:
with open('text_data.txt', 'wt') as f:
    f.write("This is the first sentence.\n")
    f.write("This is the second sentence.\n")
    f.write("Spicy Jalapeño")
f.close()

In [17]:
with open('text_data.txt', 'rt') as f:
    text_data = f.read()
f.close()
print(text_data)

This is the first sentence.
This is the second sentence.
Spicy Jalapeño


Let's try a redirected `print()` call:

In [18]:
with open('print_statement.txt', 'wt') as f:
    print("This is the first sentence.", file=f)
    print("This is the second sentence.", file=f)
f.close()

In [19]:
with open('print_statement.txt', 'rt') as f:
    print_statement = f.read()
f.close()
print(print_statement)

This is the first sentence.
This is the second sentence.



To append to the end of an existing file, use `open()` with mode `at`.  
By default, files are read/written using the system default text encoding, as can be found in `sys.getdefaultencoding()`.  
On most machines, this is set to `utf-8`.  
If you know that the text you are reading or writing is in a different encoding, supply the optional encoding parameter to `open()`.

Python understands several hundred possible text encodings.  
However, some of the more common encodings are `ascii, latin-1, utf-8, and utf-16`.  
`UTF-8` is usually a safe bet if working with web applications.  
`ascii` corresponds to the 7-bit characters in the range `U+0000` to `U+007F`.  
`latin-1` is a direct mapping of bytes 0-255 to Unicode characters `U+0000` to `U+00FF`.  
`latin-1` encoding is notable in that it will never produce a decoding error when reading text of a possibly unknown encoding.  
Reading a file as `latin-1` might not produce a completely correct text decoding, but it still might be enough to extract useful data out of it.   
Also, if you later write the data back out, the original input data will be preserved.

In [20]:
with open('text_data.txt', 'rt', encoding='latin-1') as f:
    text_data = f.read()
f.close()
print(text_data)

This is the first sentence.
This is the second sentence.
Spicy JalapeÃ±o


### Discussion

Reading and writing text files is typically very straightforward.  
However, there are a number of subtle aspects to keep in mind.  
First, the use of the `with` statement in the examples establishes a context in which the file will be used.  
<b>When control leaves the `with` block, the file will be closed automatically</b>.  
You don’t need to use the `with` statement, but if you don’t use it, make sure you remember to close the file, like so:

In [21]:
f = open('text_data.txt', 'rt')
text_data = f.read()
f.close()
print(text_data)

This is the first sentence.
This is the second sentence.
Spicy Jalapeño


Another minor complication concerns the recognition of newlines, which are different on Unix and Windows (i.e., `\n` versus `\r\n`).  
By default, Python operates in what’s known as "universal newline" mode.  
In this mode, all common newline conventions are recognized, and newline characters are converted to a single `\n` character while reading.  
Similarly, the newline character `\n` is converted to the system default newline character on output.  
If you don’t want this translation, supply the `newline=''` argument to `open()`, like this:

In [22]:
# Read a file with a disabled newline translation:
with open('text_data.txt', 'rt', newline='') as f:
    text_data = f.read()
print(text_data)

This is the first sentence.
This is the second sentence.
Spicy Jalapeño


To illustrate the difference, here’s what you will see on a Unix machine if you read the contents of a Windows-encoded text file containing the raw data `hello world!\r\n`:

A final issue concerns possible encoding errors in text files.  
When reading or writing a text file, you might encounter an encoding or decoding error.

If you get this error, it usually means that you’re not reading the file in the correct encoding.  
You should carefully read the specification of whatever it is that you’re reading and check that you’re doing it right (e.g., reading data as `UTF-8` instead of `Latin-1` or whatever it needs to be).  
If encoding errors are still a possibility, you can supply an optional `errors` argument to `open()` to deal with the errors.  
Here are two samples of common error handling schemes:

In [23]:
# Replace bad characters with Unicode U+fffd replacement characters:
with open('text_data.txt', 'rt', encoding='ascii', errors='replace') as f:
    text_data = f.read()
print(text_data)

This is the first sentence.
This is the second sentence.
Spicy Jalape��o


In [24]:
# Ignore bad characters entirely:
f = open('text_data.txt', 'rt', encoding='ascii', errors='ignore')
text_data = f.read()
f.close()
print(text_data)

This is the first sentence.
This is the second sentence.
Spicy Jalapeo


If you’re constantly fiddling with the `encoding` and `errors` arguments to `open()` and doing lots of hacks, you’re probably making life more difficult than it needs to be.  
The number one rule with text is that you simply need to make sure you’re always using the proper text encoding.  
When in doubt, use the default setting (typically UTF-8).

## [Printing to a File](http://chimera.labs.oreilly.com/books/1230000000393/ch05.html#_printing_to_a_file)

### Problem

You want to redirect the output of the `print()` function to a file.

### Solution

Use the [`file` keyword argument](https://docs.python.org/3/library/functions.html#print) to `print()`, like this:

### Discussion

There's not much more to printing to a file other than this.  
However, make sure that the file is opened in text mode.  
Printing will fail if the underlying file is in binary mode.

## Printing with a Different Separator or Line Ending